In [1]:
import cv2
import numpy as np
import os

In [3]:
cap=cv2.VideoCapture(0)
i=0
face_data=[]
path='./data/'
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

#########KNN#########
# APPLYING KNN

def dist(x1,x2):
    return(np.sqrt(sum((x1-x2)**2)))

def knn(x,y,querry,k=5):
    
    vals=[]
    m=x.shape[0]
    
    for i in range(m):
        val=dist(querry,x[i])
        vals.append((val,y[i]))
        
    vals=sorted(vals)
    vals=vals[:k]
    vals=np.array(vals)
    
    new_vals=np.unique(vals[:,1],return_counts=True)
    
    
    index=new_vals[1].argmax()
    pred=new_vals[0][index]
    
    return pred

############################################
labels=[]
class_id=0
names={}


#############data preparation#############3
for fx in os.listdir(path): ## iterating on eacch file in the path directory
    
    if fx.endswith('.npy'):
        ###creating a mapping
        names[class_id]=fx[ :-4] ## taking out name form the file name inn the directory
        print('loaded file',fx)
        data_item=np.load(path+fx) ## file read is loaded in the data_item variable return:tuple list dic stored in the file
        face_data.append(data_item)## appending the face data
        
        
        
        
        ###### creating labels for class
        target=class_id*np.ones((data_item.shape[0],)) ## providing same id to each face_data of same file
        class_id+=1
        labels.append(target)
     ### creating training dataset   
    face_dataset=np.concatenate(face_data,axis=0) #concatenatng every data from each file in dir 
    face_label=np.concatenate(labels,axis=0).reshape((-1,1))# concatenating &reshaping label   
    
while True:
    ret,frame=cap.read()
    if ret==False:
       continue
    ## for capturing faces from video 
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    ###iterating on faces  taking x,y,w,h as the faces coordinates
    for face in faces:
        x,y,w,h=face
        ## face_section: face in the video with its boundary
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        ### predicting the label by giving the training data 
        out=knn(face_dataset,face_label,face_section.flatten())
        ## finding out the names corresponding to its label in name  dict
        pred_name=names[int(out)]
        ## cv2.putText to put name or text in the frame
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        #creating rectangle on the face 
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2) 
    cv2.imshow("faces",frame)
    
    
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break

        
cap.release()
cv2.destroyAllWindows()
        
        
        
        
        
        
        
        
        
        
        
    

loaded file anju.npy
loaded file sahil.npy
loaded file sid.npy
loaded file siddhant.npy
loaded file vijay.npy
